# Data Cleaning & Feature Engineering
As the datasets provided are messy, this is very helpful to clean it before loading into database and for that python and pandas comes in power. Hence this notebook will step by step clean the given data to get it ready for loading into a database and running SQL quaeries.

## Load necessary libraries and Initial setup

In [1]:
# Load libraries
import pandas as pd
import numpy as np

# Set file path
path = "../data/"

# Load raw data files

In [2]:
# Load data files
games_raw = pd.read_csv(path + "games.csv")
sales_raw = pd.read_csv(path + "vgsales.csv")

## Inspect data

### Shape & Size

In [6]:
print("games.csv shape: ", games_raw.shape)
print("vgsales.csv shape: ", sales_raw.shape)

games.csv shape:  (1512, 14)
vgsales.csv shape:  (16598, 11)


### Column names & data types

In [8]:
games_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1512 non-null   int64  
 1   Title              1512 non-null   object 
 2   Release Date       1512 non-null   object 
 3   Team               1511 non-null   object 
 4   Rating             1499 non-null   float64
 5   Times Listed       1512 non-null   object 
 6   Number of Reviews  1512 non-null   object 
 7   Genres             1512 non-null   object 
 8   Summary            1511 non-null   object 
 9   Reviews            1512 non-null   object 
 10  Plays              1512 non-null   object 
 11  Playing            1512 non-null   object 
 12  Backlogs           1512 non-null   object 
 13  Wishlist           1512 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 165.5+ KB


In [9]:
sales_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB
